# <center>Deepseek企业级Agent项目开发实战</center>

## <center>Part 2. Microsoft GraphRAG 索引构建细节源码详解</center>

&emsp;&emsp;`Microsoft GraphRAG` 的索引构建流程，核心实现的是：<font color=red>利用大模型从传入的文档内容中提取出节点（实体）和边（关系），然后利用社区检测算法对整个知识图谱进行划分，划分成多个包含了相关性较高的节点和边的子图，然后利用大模型对每个子图进行总结，生成社区报告（摘要）（该社区报告（摘要）用来描述每个子图的概况）。</font> 当进行问答检索的时候，每个社区会依次执行`Query`的检索，最终将每个社区的结果进行汇总，从而生成全局性的完整答案。

&emsp;&emsp;其核心实现流程如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503041849610.png" width=80%></div>

&emsp;&emsp;接下来我们就针对上图的索引构建流程，进行详细的源码解析。

&emsp;&emsp;无论是使用`Microsoft GraphRAG`的`CLI`工具，还是使用`Microsoft GraphRAG`的源码应用，当运行`poetry run poe init/index/query`等命令时，其执行的入口文件是`graphrag/cli/main.py`文件。这是`Microsoft GraphRAG`所有提供功能的入口文件。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071234429.png" width=100%></div>

&emsp;&emsp;接下来我们关注`_index_cli`函数，该函数是构建索引工作流的的入口函数。

&emsp;&emsp;通过上一节课课程的快速实现流程，我们可以在控制台明确的看到构建索引的工作流所包含的9个阶段，如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071238037.png" width=90%></div>

&emsp;&emsp;构建索引的全部核心文件都集中在`graphrag/index`目录下，其源码中的`workflow`工厂函数位置如下：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071245460.png" width=90%></div>

&emsp;&emsp;我们就按照这个顺序，依次借助源码给大家进行讲解每个环节的内部实现细节。

# 1. 阶段1. 文档加载器的输入输出格式

&emsp;&emsp;上节课程中我们在快速构建索引流程中提到过，`Microsoft GraphRAG` 目前仅支持`txt`和`csv`文件格式。这是因为其内部仅仅实现了`txt`和`csv`文件的加载器，因此其他格式的文件是无法直接进行解析的。其源码位置如下：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071249775.png" width=80%></div>

&emsp;&emsp;除了关注能够解析文档的格式外，要<font color=red>明确`Microsoft GraphRAG` 构建`workflow` 的输入输出，是以 `Dataframe` 格式来进行传递的。</font>同时针对输入的文件还可以在`settings.yml`文件中，通过`input`参数来指定额外的一些实现逻辑。如下表所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Input参数说明</font></p>
<div class="center">

| 参数名称                     | 类型                     | 描述                                   |
|------------------------------|--------------------------|----------------------------------------|
| `type`                       | InputType              | 使用的输入类型。选项是`file`或`blob`                      |
| `file_type`                  | InputFileType          | 指定使用的输入文件类型，目前只支持`csv`和`txt`              |
| `base_dir`                   | str                    | 读取输入文件的根目录                |
| `connection_string`          |  str | None             | 指定 Azure Blob 存储的连接字符串      |
| `storage_account_blob_url`   | str | None            | 指定存储账户的 Blob URL                |
| `container_name`             | str | None            | 指定 Azure Blob 存储的容器名称        |
| `file_encoding`                   | str                    | 指定输入文件的编码                     |
| `file_pattern`               | str                    | 指定输入文件的模式                     |
| `file_filter`                | dict[str, str] | None | 用于匹配输入文件的正则表达式。该正则表达式必须为 file_filter 中的每个字段指定组。              |
| `text_column`                | str                    | 指定使用的输入文本列，仅限`csv`文件类型使用                   |
| `title_column`               | str | None            | 指定使用的输入标题列， 仅限`csv`文件类型使用               |
| `metadata`                   | list[str] | None      | 指定使用的文档属性列                   |

&emsp;&emsp;其中`storage_account_blob_url`、`connection_string`和`container_name`是`Azure Blob Storage`的连接字符串和容器名称，仅当使用`Microsoft Azure` 提供的云存储服务时才会用到，大家可以不必关注。

&emsp;&emsp;所有参数可以定义的位置在`settings.yml`文件中，即：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503061601765.png" width=80%></div>

&emsp;&emsp;其中这里需要重点关注的是`file_pattern` 和 `file_filter` 参数。其中`file_pattern` 需要传入的是正则表达式，用于指定输入文件的匹配模式。而`file_filter`则是需要根据`file_pattern` 的匹配结果，对匹配到的文件进行进一步的过滤。比如我有如下文件名：


```json
    "input/source_2025-03-05.txt",
    "input/source_2025-03-06.txt",
    "input/source_2025-03-07.txt"
```
&emsp;&emsp;如果想以日期作为过滤的对象，则需要通过正则去匹配`year`、`month`和`day`，然后进行过滤。示例如下：

```json
    file_pattern: '^(?P<source>[^/]+)_(?P<year>\d{4})-(?P<month>\d{2})-(?P<day>\d{2})\.txt' 
    file_filter:
        year: '2025'
        month: '03'
        day: '05'
```



&emsp;&emsp;然后在`Settings.yml`文件中，`file_pattern` 和 `file_filter` 参数的定义如下所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503070943841.png" width=100%></div>

&emsp;&emsp;配置好后在通过`poetry run poe index --root ./`命令执行索引构建时，会根据`file_pattern` 和 `file_filter` 参数的定义，对输入文件进行过滤。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503070946920.png" width=100%></div>

&emsp;&emsp;这里可以看到，尽管`input`文件夹下面有很多的`.txt`文件，但根据过滤规则其实只有`input/source_2025-03-05.txt`文件会被匹配到。巧妙应用`file_pattern` 和 `file_filter` 参数，可以实现对输入文件的灵活过滤。 

&emsp;&emsp;第二个关注点：`metadata` 参数通常是用来描述输入文件的元数据，比如文件的作者、创建时间、修改时间等。这些元数据会被存储在索引的元数据中，方便后续的查询和分析。但是在`Settings.yml`文件中的`input`参数中，`metadata`参数默认值是`none`，表示不使用元数据。 而如果想使用的话，并不是可以随意添加的，而是需要根据`txt`或者`csv`文件加载器中定义的字段来进行指定。因为`Microsoft GraphRAG`默认实现的文档加载器会把数据解析出来后转化成`Dataframe`格式。其中：`txt`和`csv`文件格式可以使用的列名是:`id`, `title`, `creation_date`, 如果在`metadata`参数中指定的话，则需要使用这些列名。



<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503061625017.png" width=100%></div>

&emsp;&emsp;但是如果使用`file_pattern` 和 `file_filter` 参数后，正则表达式中定义的列名，在`metadata`参数中是可以进行扩展指定的，比如：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503061628219.png" width=100%></div>

# 阶段2. 创建文本单元

&emsp;&emsp;准备好数据以后，才正式进入`Microsoft GraphRAG`实现的索引构建流程。其中，第一阶段要做的事情是：将传入的文档内容进行分块，然后生成`TextUnit`。`TextUnit`是用于图提取技术的文本块，同时会被提取的知识项用作源引用，以便能够溯源到最原始的文本。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071016269.png" width=60%></div>

&emsp;&emsp;其核心源码位置如下：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071306740.png" width=80%></div>

&emsp;&emsp;我们可以提取出在将原始文档切分成`TextUnit`时可以使用的一些参数，如下所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Chunk参数说明</font></p>
<div class="center">


| 字段名称                     | 参数类型               | 描述                           |
|------------------------------|------------------------|--------------------------------|
| `size`                       | `int`                  | 使用的块大小。                |
| `overlap`                    | `int`                  | 使用的块重叠量。              |
| `group_by_columns`           | `list[str]`           | 用于分块的列，默认是以 `id` 分组             |
| `strategy`                   | `ChunkStrategyType`    | 使用的分块策略。              |
| `encoding_model`             | `str`                  | 使用的编码模型。              |
| `prepend_metadata`           | `bool`                 | 是否在每个块前添加元数据。    |
| `chunk_size_includes_metadata` | `bool`                 | 在最大令牌中是否计算元数据。 |

</div>

&emsp;&emsp;其中 `size` 参数是用来指定每个块的大小，`overlap` 参数是用来指定每个块的重叠量。主要是用于控制分块的粒度。 当添加了`overlap`参数后，分块的粒度会变小，并且一定程度上可以避免按照`token`分块时，完整的句子被拆分到不同块中。较大的块会导致输出保真度较低，参考文本意义较小。但是，使用较大的块可以大大缩短处理时间。

&emsp;&emsp;`group_by_columns` 参数是用来指定分块的列，默认是以 `id` 分组。正常来说，`id` 列是唯一的，每个`.txt`或者`.csv`文件会对应一个`id`。如果以`id`分组，那么`size`和`overlap`参数会以单个文件为单位进行分块。当文档很短并且我们需要其中几个来组成一个有意义的分析单元（例如推文或聊天记录）时，这个就非常关键。

&emsp;&emsp;为了进行测试。将`file_filter`中的`day`过滤去掉，把三篇文档都加载进来。同时调整`size`参数为`300`，`overlap`参数为`100`。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503070958546.png" width=100%></div>

&emsp;&emsp;执行命令后，可以看到`input`文件夹下的三篇文档是完全按照每个`id`为单位进行分块的。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071002376.png" width=100%></div>

&emsp;&emsp;此时，我们也可以通过`group_by_columns`参数指定以某个类为单位进行分块，比如这里我们以`month`为单位进行分块。如下：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071004070.png" width=80%></div>

&emsp;&emsp;其检索情况就会如下图所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071006154.png" width=100%></div>

&emsp;&emsp;然后再看`strategy`参数，`strategy`参数可以指定为`ChunkStrategyType`类型，主要包含两种，分别是`token`和`sentence`。其中：

- tokens：基于令牌进行分块，每个块的大小为`size`参数指定的值，重叠量为`overlap`参数指定的值。
- sentence：基于`NLTK`的句子分割器进行分块，适合需要进行句子级别分析的应用，如情感分析、文本摘要等。

&emsp;&emsp;`NLTK` 的句子分割器一般是通过 `sent_tokenize` 函数实现，能够有效地将文本分割成句子。比如一个代码示例是：

```python
    import nltk
    from nltk.tokenize import sent_tokenize

    # 下载 punkt 模型（如果尚未下载）
    nlt k.download('punkt')

    # 示例中文文本
    text = "你好！你最近怎么样？这是一个句子分割的示例。让我们看看它是如何工作的。"

    # 使用 sent_tokenize 进行句子分割
    sentences = sent_tokenize(text|, language='chinese')

    # 输出分割后的句子
    for sentence in sentences:
        print(sentence)
```

&emsp;&emsp;输出文本：

```
    你好！
    你最近怎么样？
    这是一个句子分割的示例。
    让我们看看它是如何工作的。
```


&emsp;&emsp;默认是`token`策略，加载我们之前设置的`size`和`overlap`参数。而如果想设置成`sentence`策略，则需要设置`strategy`参数为`sentence`，如下所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071013292.png" width=80%></div>

&emsp;&emsp;当执行完第一个`create_base_text_units` 的`workflow`后，在`output`文件夹下会生成`documents.parquet`文件和`text_units.parquet`文件。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071316357.png" width=80%></div>

&emsp;&emsp;我们可以通过`pandas`库来依次读取这两个文件，并查看其内容。首先安装依赖：

In [6]:
# 安装两个包，其中 tabulate 是用来格式化输出，pandas 是用来读取 parquet 文件

! pip install tabulate
! pip install pandas

&emsp;&emsp;然后读取`text_units.parquet`文件和`documents.parquet`文件，并查看其内容。

In [9]:
import pandas as pd

# 使用绝对路径读取 Parquet 文件
df_documents = pd.read_parquet('documents.parquet')  # 替换为实际路径
df_text_units = pd.read_parquet('text_units.parquet')  # 替换为实际路径

In [10]:
# 显示documents.parquet文件的前5行
print(df_documents.head(5))

                                 source  year month day  \
0  E:\my_graphrag\graphrag\input\source  2025    03  05   
1  E:\my_graphrag\graphrag\input\source  2025    03  06   
2  E:\my_graphrag\graphrag\input\source  2025    03  07   

                                                text  \
0  在过去的几十年中，全球科技行业经历了翻天覆地的变化。以硅谷为中心的创新生态系统催生了许多世界...   
1                                                测试2   
2                                                测试3   

                                                  id                  title  \
0  cd6425af96f68daa671cdf748fc4850b4c99dff7f291fb...  source_2025-03-05.txt   
1  156dd88a2f6d45caebc4a2455f55cc2716d03bcc9e454e...  source_2025-03-06.txt   
2  9d6114a8bdc915ce041695258538329c94d66dedcc5da9...  source_2025-03-07.txt   

               creation_date         metadata  
0  2025-03-06 15:36:51 +0800  {'month': '03'}  
1  2025-03-06 15:36:07 +0800  {'month': '03'}  
2  2025-03-06 15:36:30 +0800  {'month': '03'}  


In [11]:
# 展示 text_units.parquet 文件的前5行
print(df_text_units.head(5))

  month                                               text  \
0    03  测试2测试3在过去的几十年中，全球科技行业经历了翻天覆地的变化。以硅谷为中心的创新生态系统催...   
1    03  1976 年在美国加利福尼亚州库比蒂诺的一个车库中创立。最初，苹果公司专注于个人电脑的开发，...   
2    03  苹果也因此成为全球最有价值的公司之一。截至 2023 年，苹果的市值已超过 2 万亿美元。\...   
3    03  �集资金超过 16 亿美元。\n\n谷歌的创新能力不仅限于搜索。2006 年，谷歌收购了 Y...   
4    03  保罗·艾伦（Paul Allen）于 1975 年在美国华盛顿州雷德蒙德创立。微软最初以开发...   

                                                  id  \
0  6f1b8ab576aa51f5c3b0732abe0ba6513c18b600ec19b7...   
1  7fc7ff44c074c1c5c050c8ad6d935d5185dee9bb088a39...   
2  6f94edfde89a353bcc7b3b9a962e43f1848c90f9d76624...   
3  20894d97f011bbcabe89a35e70a7ae7d04a87ac995f59a...   
4  1154060a589e9036c32f89c931bfc968fd34864e4ff359...   

                                        document_ids  n_tokens  
0  [156dd88a2f6d45caebc4a2455f55cc2716d03bcc9e454...       300  
1  [cd6425af96f68daa671cdf748fc4850b4c99dff7f291f...       300  
2  [cd6425af96f68daa671cdf748fc4850b4c99dff7f291f...       300  
3  [cd6425af96f68daa671cdf748f

&emsp;&emsp;然后读取`documents.parquet`文件，并查看其内容。

&emsp;&emsp;最后，使用`tabulate`库来格式化输出`documents.parquet`文件和`text_units.parquet`文件的内容。

In [12]:
from tabulate import tabulate

# 其中，headers='keys' 表示使用列名作为表头，tablefmt='pretty' 表示使用 pretty 格式化输出，showindex=False 表示不显示行索引，stralign='left' 表示左对齐，maxcolwidths=[20, 20, 20, 20, 20] 表示每列的最大宽度为20
print(tabulate(df_documents, headers='keys', tablefmt='pretty', showindex=False, stralign='left', maxcolwidths=[20, 20, 20, 20, 20]))

+----------------------+------+-------+-----+----------------------+----------------------------------------------------------------------------------------------------------------------------------+-----------------------+---------------------------+-----------------+
| source               | year | month | day | text                 | id                                                                                                                               | title                 | creation_date             | metadata        |
+----------------------+------+-------+-----+----------------------+----------------------------------------------------------------------------------------------------------------------------------+-----------------------+---------------------------+-----------------+
| E:\my_graphrag\graph | 2025 | 03    | 05  | 在过去的几十年中，全 | cd6425af96f68daa671cdf748fc4850b4c99dff7f291fbf9ab774e533312124e2c82d42c8c8f9775fa1d222358b29ffa85ce10d87bd9566d6dcbd5d9ad00b1bc | 

In [13]:
from tabulate import tabulate

# 其中，headers='keys' 表示使用列名作为表头，tablefmt='pretty' 表示使用 pretty 格式化输出，showindex=False 表示不显示行索引，stralign='left' 表示左对齐，maxcolwidths=[20, 20, 20, 20, 20] 表示每列的最大宽度为20
print(tabulate(df_text_units, headers='keys', tablefmt='pretty', showindex=False, stralign='left', maxcolwidths=[20, 20, 20, 20, 20]))

+-------+----------------------+----------------------+----------------------+----------+
| month | text                 | id                   | document_ids         | n_tokens |
+-------+----------------------+----------------------+----------------------+----------+
| 03    | 测试2测试3在过去的几 | 6f1b8ab576aa51f5c3b0 | ['156dd88a2f6d45caeb | 300      |
|       | 十年中，全球科技行业 | 732abe0ba6513c18b600 | c4a2455f55cc2716d03b |          |
|       | 经历了翻天覆地的变化 | ec19b754553e15a004ff | cc9e454e1156e7cf20ca |          |
|       | 。以硅谷为中心的创新 | 4d11b638b85c25d8e12a | e46c130433038eeff9b9 |          |
|       | 生态系统催生了许多世 | 50501eb882ec711f8fa9 | 7b5424d63dddb1a30072 |          |
|       | 界知名的科技公司，如 | f5df17ca7a0bb3f18522 | 354f2a59164209487fe6 |          |
|       | 苹果、谷歌、微软和亚 | 0f96e38e             | 09e136b30a'  '9d6114 |          |
|       | 马逊。这些公司不仅改 |                      | a8bdc915ce0416952585 |          |
|       | 变了人们的生活方式， |                      | 38329c94d66dedcc5da9 |          |
|       |

&emsp;&emsp;当得到这两个文档以后，会执行`create_final_documents` 的`workflow`，将这两个文档的字段内容进行合并，即把`text_units.parquet`文件中的字段合并到`documents.parquet`文件中。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071045413.png" width=80%></div>

&emsp;&emsp;同时，在`output`文件夹下的`documents.parquet`文件会进行更新，关联到`text_units.parquet`文件中的`id`列。

In [15]:
new_df_documents = pd.read_parquet('documents.parquet')  # 替换为实际路径
# 假设 df 是你的 DataFrame
print(tabulate(new_df_documents, headers='keys', tablefmt='pretty', showindex=False, stralign='left', maxcolwidths=[20, 20, 20, 20, 20]))

+----------------------+-------------------+----------------------+----------------------+----------------------+---------------------------+-----------------+
| id                   | human_readable_id | title                | text                 | text_unit_ids        | creation_date             | metadata        |
+----------------------+-------------------+----------------------+----------------------+----------------------+---------------------------+-----------------+
| cd6425af96f68daa671c | 1                 | source_2025-03-05.tx | 在过去的几十年中，全 | ['6f1b8ab576aa51f5c3 | 2025-03-06 15:36:51 +0800 | {'month': '03'} |
| df748fc4850b4c99dff7 |                   | t                    | 球科技行业经历了翻天 | b0732abe0ba6513c18b6 |                           |                 |
| f291fbf9ab774e533312 |                   |                      | 覆地的变化。以硅谷为 | 00ec19b754553e15a004 |                           |                 |
| 124e2c82d42c8c8f9775 |                   |                      | 中心

# 阶段3. 图元素的提取

&emsp;&emsp;在对文档进行分块成`TextUnit`以后，接下来就要对每个`TextUnit`中的内容进行图元素的提取。 元素主要包括：实体、关系。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071034125.png" width=80%></div>

&emsp;&emsp;其源码中对应的工作流入门如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071332417.png" width=80%></div>

&emsp;&emsp;主要有两种提取策略：

- 默认的提取策略是：`graph_intelligence`，其实就是编写提示词让大模型来提取出实体和关系。
- 另一种是使用`NLTK`进行实体和关系的提取。

&emsp;&emsp;这里建议大家还是选择使用大模型来进行实体和关系的提取，因为大模型提取的效果会更好。自然，这个环节需要加载我们在`settings.yml`文件中配置的对话模型接口服务。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071338847.png" width=80%></div>

&emsp;&emsp;同时，在`settings.yml`文件中，我们还可以配置实体的类型和提示词，位置如下图所示：

&emsp;&emsp;默认的实体类型是："organization", "person", "geo", "event"

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071108656.png" width=80%></div>

&emsp;&emsp;默认的实体类型是："organization", "person", "geo", "event", 而提示词内容就是在初始化`graphrag`项目时，在`promots`文件夹下的`extract_graph.txt`文件中定义的。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071342318.png" width=80%></div>

&emsp;&emsp;这里的问题是：不同的数据块（chunks）可能会抽取出相同的实体。`Microsoft GraphRAG`会采用`merge`操作，如果遇到相同的节点，那么`GraphRAG`就会执行`concat`操作，也就是将对应的属性和关系进行合并。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071349938.png" width=80%></div>

&emsp;&emsp;同时，对节点和关系分别基于`TextUnit`的原始文本进行摘要。简单理解就是给出一个解释：为什么这个节点和关系会被提取出来。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071353423.png" width=80%></div>

&emsp;&emsp;同时，也可以在`settings.yml`文件中进行更改。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071355439.png" width=80%></div>

&emsp;&emsp;这就是第三个阶段的完整流程，即从`TextUnit`中提取出实体和关系以及生成对应的摘要。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071121796.png" width=100%></div>

&emsp;&emsp;执行`extract_graph`的`workflow`后，在`output`文件夹下会生成`entities.parquet`文件和`relations.parquet`文件。可以用同样的方式来进行读取。


In [16]:
# 使用绝对路径读取 Parquet 文件
df_entities = pd.read_parquet('entities.parquet')  # 替换为实际路径
df_relations = pd.read_parquet('relationships.parquet')  # 替换为实际路径

In [18]:
print(tabulate(df_entities, headers='keys', tablefmt='pretty', showindex=False, stralign='left', maxcolwidths=[20, 20, 20, 20, 20]))

+----------------------+--------------+----------------------+-----------+----------------------+
| title                | type         | text_unit_ids        | frequency | description          |
+----------------------+--------------+----------------------+-----------+----------------------+
| APPLE INC.           | ORGANIZATION | ['6f1b8ab576aa51f5c3 | 1         | Apple Inc.是一家由史 |
|                      |              | b0732abe0ba6513c18b6 |           | 蒂夫·乔布斯、史蒂夫· |
|                      |              | 00ec19b754553e15a004 |           | 沃兹尼亚克和罗恩·韦  |
|                      |              | ff4d11b638b85c25d8e1 |           | 恩于1976年在美国加利 |
|                      |              | 2a50501eb882ec711f8f |           | 福尼亚州库比蒂诺的一 |
|                      |              | a9f5df17ca7a0bb3f185 |           | 个车库中创立的科技公 |
|                      |              | 220f96e38e']         |           | 司，专注于个人电脑开 |
|                      |              |                      |           | 发。    

In [19]:
print(tabulate(df_relations, headers='keys', tablefmt='pretty', showindex=False, stralign='left', maxcolwidths=[20, 20, 20, 20, 20]))

+----------------+----------------------+----------------------+--------+----------------------+
| source         | target               | text_unit_ids        | weight | description          |
+----------------+----------------------+----------------------+--------+----------------------+
| APPLE INC.     | STEVE JOBS           | ['6f1b8ab576aa51f5c3 | 8.0    | Steve Jobs是Apple In |
|                |                      | b0732abe0ba6513c18b6 |        | c.的创始人之一，并对 |
|                |                      | 00ec19b754553e15a004 |        | 公司的创新和发展起到 |
|                |                      | ff4d11b638b85c25d8e1 |        | 了关键作用。         |
|                |                      | 2a50501eb882ec711f8f |        |                      |
|                |                      | a9f5df17ca7a0bb3f185 |        |                      |
|                |                      | 220f96e38e']         |        |                      |
| APPLE INC.     | STEVE WOZNIAK        | ['6f1b8ab576aa

&emsp;&emsp;同样，在提取出实体和关系后，会执行`finalize_graph`的，分别对`entities.parquet`文件和`relations.parquet`文件进行进一步处理，将执行`finalize_graph`的`workflow`。在这个过程中，将执行图节点和关系的聚合，并生成图的向量表示。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071418645.png" width=80%></div>

In [20]:
import pandas as pd

# 使用绝对路径读取 Parquet 文件
df_entities = pd.read_parquet('entities.parquet')  # 替换为实际路径
df_relations = pd.read_parquet('relationships.parquet')  # 替换为实际路径

In [21]:
print(tabulate(df_entities, headers='keys', tablefmt='pretty', showindex=False, stralign='left', maxcolwidths=[20, 20, 20, 20, 20]))

+----------------------+-------------------+----------------------+--------------+----------------------+--------------------------------------------------------------------------------------------------------------------------------------+-----------+--------+-----+-----+
| id                   | human_readable_id | title                | type         | description          | text_unit_ids                                                                                                                        | frequency | degree | x   | y   |
+----------------------+-------------------+----------------------+--------------+----------------------+--------------------------------------------------------------------------------------------------------------------------------------+-----------+--------+-----+-----+
| 0274a442-8706-48ce-8 | 0                 | APPLE INC.           | ORGANIZATION | Apple Inc.是一家由史 | ['6f1b8ab576aa51f5c3b0732abe0ba6513c18b600ec19b754553e15a004ff4d11b638b85c25d

In [22]:
print(tabulate(df_relations, headers='keys', tablefmt='pretty', showindex=False, stralign='left', maxcolwidths=[20, 20, 20, 20, 20]))

+----------------------+-------------------+----------------+----------------------+----------------------+--------+-----------------+--------------------------------------------------------------------------------------------------------------------------------------+
| id                   | human_readable_id | source         | target               | description          | weight | combined_degree | text_unit_ids                                                                                                                        |
+----------------------+-------------------+----------------+----------------------+----------------------+--------+-----------------+--------------------------------------------------------------------------------------------------------------------------------------+
| 973803f1-871f-4739-9 | 0                 | APPLE INC.     | STEVE JOBS           | Steve Jobs是Apple In | 8.0    | 8               | ['6f1b8ab576aa51f5c3b0732abe0ba6513c18b600ec19b754553e15

# 阶段4. 社区检测

&emsp;&emsp;现在有了可用的实体和关系图，但是这些实体和关系都是孤立的，没有形成一个完整的图谱。因此需要做的就是将这些实体和关系进行聚合，形成一个完整的图谱。所以接下来的任务就是要将识别出来的实体和关系分组成相关关联的子集。 

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071137479.png" width=80%></div>

&emsp;&emsp;要实现这个目标，就需要大家理解社区检测这个概念。

&emsp;&emsp;社区检索是图论中的一个重要任务，旨在识别图中节点的聚集结构。其中社区是指在图中，节点之间的连接比与其他节点的连接更为密切的子集。通过识别社区，我们就可以理解数据的内在结构，发现潜在的模式和关系。其中莱顿算法（Leiden Algorithm）是一种用于社区检测的高效算法，旨在优化社区结构的识别过程。它是基于 Louvain 算法的改进，具有更好的性能和准确性。

&emsp;&emsp;微软实现的就是分层的莱顿算法（Hierarchical Leiden Algorithm），它将图中的节点分成多个层次的社区，从而形成一个层次化的社区结构。

&emsp;&emsp;`Microsoft GraphRAG` 使用的莱顿算法是基于`graspologic`库的实现。它主要做的是对`Nodes`去划分社区，完成后，会生成`communities.parquet`文件，该文件中会包含每个社区的属性信息。其核心实现位置如下：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071621829.png" width=80%></div>

&emsp;&emsp;其中社区划分`workflow`为`create_communities`， 其源码位置如下：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071626228.png" width=80%></div>

&emsp;&emsp;该过程可以调整的策略不多，仅有一个参数用来控制划分出多少个社区。同样可以在`setting.yml`中进行配置，如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071628993.png" width=80%></div>

&emsp;&emsp;然后再次执行`poetey run poe index --root ./`后，可以得到`communities.parquet`文件。依然可以使用相同的方式进行读取查看。

In [25]:
# 使用绝对路径读取 Parquet 文件
df_communities = pd.read_parquet('communities.parquet')  # 替换为实际路径

In [27]:
print(tabulate(df_communities, headers='keys', tablefmt='pretty', showindex=False, stralign='left', maxcolwidths=[20, 20, 20, 20, 20]))

+----------------------+-------------------+-----------+-------+--------+----------+-------------+------------------------------------------+------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+------------+------+
| id                   | human_readable_id | community | level | parent | children | title       | entity_ids                               | relationship_ids                         | text_unit_ids                                                                                                                        | period     | size |
+----------------------+-------------------+-----------+-------+--------+----------+-------------+------------------------------------------+------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------+-

&emsp;&emsp;莱顿算法（Louvain Method）通过优化模块度（modularity）来划分图中的节点，并没有固定的默认划分数量。它的目标是根据图的结构自动识别社区的数量。算法会在每个 `level` 中优化社区划分，直到达到局部最优。每个 `level` 代表一个图的状态，随着算法的迭代，图的结构会逐渐简化，节点会被聚合成更大的社区。

&emsp;&emsp;在生成社区后，执行的`workflow`是`create_final_text_units`，其实就是把之前处理的实体、关系等关联起来，形成最终的文本单元。形成一个完整的知识图谱表示，这样，每个文本单元不仅包含原始文本，还包含了与之相关的所有结构化信息（实体、关系）的引用，可以用于进一步 `Embedding` 和 后续的`Query`操作。


In [28]:
df_text_units = pd.read_parquet('text_units.parquet')  # 替换为实际路径


print(tabulate(df_text_units, headers='keys', tablefmt='pretty', showindex=False, stralign='left', maxcolwidths=[20, 20, 20, 20, 20]))

+----------------------+-------------------+----------------------+----------+----------------------+------------------------------------------+------------------------------------------+---------------+
| id                   | human_readable_id | text                 | n_tokens | document_ids         | entity_ids                               | relationship_ids                         | covariate_ids |
+----------------------+-------------------+----------------------+----------+----------------------+------------------------------------------+------------------------------------------+---------------+
| 6f1b8ab576aa51f5c3b0 | 1                 | 测试2测试3在过去的几 | 300      | ['156dd88a2f6d45caeb | ['cf9c2e52-d0ca-42bd-bb55-b8d95f0c3c2d'  | ['abf5b642-5f5f-4dfb-86f2-35ecc2722237'  | []            |
| 732abe0ba6513c18b600 |                   | 十年中，全球科技行业 |          | c4a2455f55cc2716d03b |  '3edac734-f18a-4aae-a317-68b6cfd8f894'  |  'ba1d6351-2d3b-4841-935f-05943e922277'  |               |

# 阶段5. 生成社区报告

&emsp;&emsp;接下来，要做的就是对每个社区中的节点、关系和摘要的定义进行总结。这样做的目的是为了方便查询，当查询时需要根据问题匹配知识库中的实体信息和关系信息时，只需要根据总结后的实体描述和关系描述就可以进行匹配了。不然遍历所有的`Description`进行匹配，效率会非常低下。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071636612.png" width=80%></div>

&emsp;&emsp;这里执行的`Workflow`就是`create_community_reports.py`，其源码如下：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071642989.png" width=80%></div>

&emsp;&emsp;同时，生成报告的策略也是借助提示工程 + 大模型实现，同时复用 实体、关系提取的部分子逻辑，因此，我们也可以在`setting.yml` 文件中进行相关提示词的配置更改：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071642990.png" width=80%></div>

&emsp;&emsp;然后再次执行`poetey run poe index --root ./`后，可以得到`community_reports.parquet`文件。依然可以使用相同的方式进行读取查看。

In [31]:
df_community_reports = pd.read_parquet('community_reports.parquet')  # 替换为实际路径

print(tabulate(df_community_reports, headers='keys', tablefmt='pretty', showindex=False, stralign='left', maxcolwidths=[20, 20, 20, 20, 20]))


+----------------------+-------------------+-----------+-------+--------+----------+------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------

In [32]:
df_community_reports = pd.read_parquet('community_reports.parquet')  # 替换为实际路径

print(tabulate(df_community_reports, headers='keys', tablefmt='pretty', showindex=False, stralign='left', maxcolwidths=[20, 20, 20, 20, 20]))


+----------------------+-------------------+-----------+-------+--------+----------+------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------

&emsp;&emsp;最后，在最新版本的代码实现中，对各个阶段不在单独生成`Embedding`向量，而是在完成所有的索引构建流程且得到索引文件后，一次性的将实体、关系、社区报告等关键信息的`Embedding`向量表示生成，并在本地生成`Lancedb`存储，用于接下来的`Query`流程。其源码位置如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202503071648290.png" width=80%></div>

&emsp;&emsp;至此，这就是完成的`Microsoft GraphRAG`构建索引的内部细节。而关于`Query`阶段，我们将在接下来的课程中进行详细的讲解，从而构建出完整的基于`GraphRAG`的问答流程。